In [6]:
import sys
sys.path.append("V:/biomoni/") 
from biomoni import Experiment
from BacillusScripts.BacillusVariableFeedrate_copy import Bacillus_vf
from biomoni import Model
import scipy
from scipy.interpolate import interp1d
import pandas as pd
import numpy as np

In [7]:
path = r"V:/biomoni/BacillusData/Stamm185"
b = Bacillus_vf()
a = Experiment(path,"F5",endpoint = "F_end" )
y0 = b.create_y0(a)
c = b.create_controls(a)

t_grid = np.linspace(0,40,1001) 
experiment_dict_for_estimation = {exp : Experiment(path, exp,endpoint = "Sim_end") for exp in ["F5"]}  #all experiments in a dictionary   
experiment_dict_for_graphs = {exp : Experiment(path, exp, endpoint = "F_end") for exp in ["F5"]} 

b.estimate(experiment_dict_for_estimation)
b.report()
print(b.p)
t_grid = np.linspace(0,40,1001) 
sim_dict_all = {experiment.exp_id: b.simulate(experiment = experiment, t_grid = t_grid) for experiment in experiment_dict_for_estimation.values()} 

print(sim_dict_all)

{'off': {'Glucose [g/L]': 0.03191800750123827, 'CDW_calc': 0.8499599165745977, 'RF [mg/L]': 0.00864467403342319}, 'on': {('BASET_2', 'Value'): 8.144595812061279e-05, ('pO2_2', 'Value'): 6.188072860845093e-05}, 'CO2': {'CO2': 0.008650996108133127}}
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 21
    # data points      = 24
    # variables        = 3
    chi-square         = 0.18045963
    reduced chi-square = 0.00859332
    Akaike info crit   = -111.367248
    Bayesian info crit = -107.833087
[[Variables]]
    Km:      0.1 (fixed)
    Yxs:     0.15908227 +/- 0.01933675 (12.16%) (init = 0.1)
    Ypx:     107.557285 +/- 4.29842202 (4.00%) (init = 100)
    HX:      1.77 (fixed)
    OX:      0.54 (fixed)
    NX:      0.16 (fixed)
    mu_max:  0.19458865 +/- 0.00136795 (0.70%) (init = 0.2)
[[Correlations]] (unreported correlations are < 0.100)
    C(Ypx, mu_max) = -0.736
    C(Yxs, mu_max) =  0.222
    C(Yxs, Ypx)    = -0.176
Parameters([('Km', <Parameter 'Km'

In [17]:
t_grid = list(a.dataset["off"].index.values)

Carb_RR = []
V = 0.5
Mw_bm = 24.445          #g/mol molecular weight of biomass assuming composition of C=1,H=1.594,N=0.293,O=0.387,P=0.012,S=0.005
Mw_gluc = 180.156       #g/mol molecular weight of glucose C=6,H=12,O=6
Mw_RF = 376.36          #g/mol molecular weight of Riboflavin C=17,H=20,N=4,O=6
dV_gas_dt = c["gas_flow"]
R = 0.08314 #bar*l/mol*K
T = c["T"] + 273.15     #Kelvin  
pressure = c["pressure"] #bar
Mw_CO2 = 44.01

F_C_in = c["feedrate_glc"](t_grid)
df = pd.DataFrame(data=F_C_in,index=t_grid, columns=["feedrate pure glucose g/h"])

# calculate cumulated amount of Glucose from Feed
F_C_in_cum = []

for t in t_grid:
    func = c["feedrate_glc"]
    cummulation = scipy.integrate.quad(func,0,t,limit=200)
    
    F_C_in_cum.append(cummulation)

df["Glc g_cum"] = [x[0] for x in F_C_in_cum]


#creating column with calculcated Volume values for CO2 [L] from exhaust gas measurment
a.dataset["CO2"]["V_CO2 L"] = (a.dataset["CO2"]["CO2"]- 0.04)/100 * 30   #30 L/h is F_exhaust or Outlet flow (assuming = inlet flow)

#creating column with cumulated volume values for CO2 [L] 
a.dataset["CO2"]["cum_CO2 L"] = scipy.integrate.cumtrapz(a.dataset["CO2"]["V_CO2 L"],a.dataset["CO2"]["V_CO2 L"].index, initial=0)




CO2_func = interp1d(x = a.dataset["CO2"].index, 
                    y = a.dataset["CO2"]["CO2"], fill_value = (a.dataset["CO2"]["CO2"].iloc[0], a.dataset["CO2"]["CO2"].iloc[-1]) , 
                    bounds_error= False)

V_CO2_func = interp1d(x = a.dataset["CO2"].index, 
                    y = a.dataset["CO2"]["V_CO2 L"], fill_value = (a.dataset["CO2"]["V_CO2 L"].iloc[0], a.dataset["CO2"]["V_CO2 L"].iloc[-1]) ,
                    bounds_error= False)

V_CO2_cum_func= interp1d(x = a.dataset["CO2"].index, 
                    y = a.dataset["CO2"]["cum_CO2 L"], fill_value = (a.dataset["CO2"]["cum_CO2 L"].iloc[0], a.dataset["CO2"]["cum_CO2 L"].iloc[-1]) ,
                    bounds_error= False)


df["CO2 %"] = CO2_func(t_grid)
df["CO2 L"] = V_CO2_func(t_grid)
df["cum_CO2 L"] = V_CO2_cum_func(t_grid)


#calculate cumulated amount of CO2 in exhaust gas 
df["CO2 mol_cum"] = (df["cum_CO2 L"] * pressure) / (R * T)



Carb_RR= {}
Carb_RR["t"]= t_grid
Carb_RR["given"]=[]
Carb_RR["found"]=[]
Carb_RR["RR"]=[]

for t in t_grid:
    
    #mass balance of Carbon given (Substrate,Biomass,Product)
    nC_Sg = y0[1]/Mw_gluc *6 + df["Glc g_cum"].values[t_grid.index(t)]/Mw_gluc * 6   #initial Substrate + Substrate from Feed
    nC_Xg = y0[0]/Mw_bm
    nC_Pg = y0[2]/Mw_RF *17
    
    amountC_given = nC_Sg + nC_Xg + nC_Pg 
    
    
    
    #mass balance of Carbon found (Substrate,Biomass,Product,CO2)
    nC_Sf = (float(a.dataset["off"]["Glucose [g/L]"].loc[[t]])*V)/Mw_gluc * 6
    nC_Xf = (float(a.dataset["off"]["CDW_calc"].loc[[t]])*V)/ Mw_bm
    nC_Pf = (float(a.dataset["off"]["RF [mg/L]"].loc[[t]])*V /1000)/ Mw_RF * 17
    nC_CO2 = float(df["CO2 mol_cum"].values[t_grid.index(t)])

    amountC_found = nC_Sf + nC_Xf + nC_Pf + nC_CO2
    
    RR = (amountC_found / amountC_given) * 100
    
    Carb_RR["given"].append(amountC_given)
    Carb_RR["found"].append(amountC_found)
   
    Carb_RR["RR"].append(RR)
 
df2 = pd.DataFrame.from_dict(Carb_RR)
 

df["RR"] = df2["RR"].values 

C:\Users\walkyre\AppData\Local\Temp/ipykernel_14952/1676391735.py:22: IntegrationWarning:

The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.

C:\Users\walkyre\AppData\Local\Temp/ipykernel_14952/1676391735.py:22: IntegrationWarning:

The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.



In [22]:
# print(a.dataset["CO2"])
# print(df)
print(df2)

            t     given     found          RR
0    0.069444  0.519105  0.509704   98.189111
1   16.069444  0.519105  0.520667  100.301084
2   17.752778  0.519105  0.508007   97.862178
3   18.586111  0.585616  0.529370   90.395334
4   19.836111  0.680799  0.584742   85.890589
5   20.802778  0.760269  0.631329   83.040233
6   21.702778  0.838818  0.690116   82.272411
7   22.852778  0.945658  0.795163   84.085750
8   39.452778  3.140775  1.672858   53.262598
9   40.719444  3.325511  1.827666   54.958935
10  41.952778  3.505382  1.883446   53.730126
11  43.402778  3.718913  1.961470   52.743116
12  44.736111  3.911276  2.020074   51.647446
13  46.052778  4.103299  2.020601   49.243335
14  47.336111  4.270620  2.096314   49.086875
15  63.719444  5.908437  2.192902   37.114759


In [19]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

x = df.index
x2 = a.dataset["CO2"].index

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter( x= x, y=df["CO2 %"],mode="markers", marker= dict(size= 12, symbol="x"),name="CO2 [%] interp1d"), secondary_y=False)

fig.add_trace(go.Scatter(x =x2, y= a.dataset["CO2"]["CO2"], name="CO2 [%] online df"))
fig.add_trace(go.Scatter(x=x,y=df["cum_CO2 L"],mode="markers", marker= dict(size= 12, symbol="x"), name ="cum_CO2 [L] integrated in online df, turned in inter1pd " ),secondary_y=True)

fig.add_trace(go.Scatter(x=x2,y=a.dataset["CO2"]["cum_CO2 L"], name ="cum_CO2 [L] integrated in online df" ),secondary_y=True)
fig.update_layout(title_text="F5 ", title_x= 0.5,titlefont=dict(size=30))
fig.update_xaxes(title_text="F-time [h]")
fig.update_yaxes(title_text="CO2[%]", secondary_y =False, titlefont=dict(size=15))
fig.update_yaxes(title_text="cumulated CO2 [L]", secondary_y =True, titlefont=dict(size=15))
fig.show()